# Лабораторная работа №1
имя : Саид Наваф  Группа:К3341   ПОТОК:1.2   ВАРИАНТ:18

 Цель работы
 Закрепить навыки постановки и решения задач линейного программирования (ЗЛП):
 - Приведение задачи к каноническому виду  
 - Построение вспомогательной задачи  
 - Решение с помощью симплекс-метода  
 - Сравнение с результатами Excel (надстройка "Поиск решения")


 1. Импорт библиотек


In [ ]:
import re
from pathlib import Path
from copy import deepcopy
from pprint import pprint
import numpy as np
import pandas as pd


 2. Функции для чтения и подготовки данных
 Здесь реализована функция `parse_file()` — она считывает задачу ЛП из текстового файла,
а также проверяет корректность данных.


In [ ]:
def parse_file(path: str):
    """Считывает задачу ЛП из текстового файла"""
    lines = [line.strip() for line in Path(path).read_text(encoding='utf-8').splitlines() if line.strip()]
    if len(lines) < 3:
        raise ValueError("Файл должен содержать минимум 3 строки: MIN/MAX, коэффициенты, число ограничений.")

    sense = lines[0].lower()
    if sense not in ("min", "max"):
        raise ValueError("Первая строка должна быть 'MIN' или 'MAX'.")

    objective = [float(x) for x in lines[1].split()]
    n_constraints = int(lines[2])

    constraints = []
    for i in range(3, 3 + n_constraints):
        line = lines[i]
        match = re.search(r'(<=|>=|=)', line)
        if not match:
            raise ValueError(f"В строке ограничения отсутствует знак (<=, >=, =): {line}")
        rel = match.group(1)
        left, right = line.split(rel)
        coeffs = [float(x) for x in left.split()]
        rhs = float(right.strip())
        constraints.append({"coeffs": coeffs, "sense": rel, "rhs": rhs})

    nvars = len(objective)
    for c in constraints:
        if len(c["coeffs"]) != nvars:
            raise ValueError("Количество коэффициентов в ограничении не совпадает с числом переменных.")

    return {"sense": sense, "objective": objective, "constraints": constraints, "nvars": nvars}

3. Приведение задачи к каноническому виду
 - преобразование `MAX` → `MIN` (при необходимости),
- добавление дополнительных переменных для ограничений.


In [ ]:
def to_canonical_form(lp_data):
    data = deepcopy(lp_data)
    sense = data["sense"]
    objective = data["objective"]
    constraints = data["constraints"]
    nvars = data["nvars"]

    if sense == "max":
        objective = [-c for c in objective]
        sense = "min"

    slack_count = 0
    new_constraints = []

    for con in constraints:
        coeffs = con["coeffs"].copy()
        sign = con["sense"]

        if sign == "<=":
            coeffs.extend([0.0] * slack_count + [1.0])
            slack_count += 1
        elif sign == ">=":
            coeffs.extend([0.0] * slack_count + [-1.0])
            slack_count += 1
        elif sign == "=":
            coeffs.extend([0.0] * slack_count)
        else:
            raise ValueError(f"Неизвестный знак ограничения: {sign}")

        new_constraints.append({"coeffs": coeffs, "rhs": con["rhs"], "sense": "="})

    max_len = max(len(c["coeffs"]) for c in new_constraints)
    for c in new_constraints:
        c["coeffs"].extend([0.0] * (max_len - len(c["coeffs"])))
    objective.extend([0.0] * (max_len - len(objective)))

    return {"sense": sense, "objective": objective, "constraints": new_constraints, "nvars": max_len}

 4. Учёт отрицательных переменных и построение вспомогательной задачи
 Функции:
 - `expand_variables_for_negativity()` — делает замену `xi = xi+ - xi-`;
 - `build_auxiliary_problem()` — формирует вспомогательную задачу для первого этапа симплекс-метода.


In [ ]:
def expand_variables_for_negativity(lp_data):
    data = deepcopy(lp_data)
    n = data["nvars"]
    new_objective = []
    new_constraints = []

    for c in data["objective"]:
        new_objective.extend([c, -c])

    for con in data["constraints"]:
        new_coeffs = []
        for a in con["coeffs"]:
            new_coeffs.extend([a, -a])
        new_constraints.append({
            "coeffs": new_coeffs,
            "sense": con["sense"],
            "rhs": con["rhs"]
        })

    data["objective"] = new_objective
    data["constraints"] = new_constraints
    data["nvars"] = n * 2
    data["var_mapping"] = [f"x{i+1}+" if j % 2 == 0 else f"x{i+1}-"
                           for i in range(n) for j in range(2)]
    return data

def build_auxiliary_problem(canonical_data):
    data = deepcopy(canonical_data)
    constraints = data["constraints"]
    nvars = data["nvars"]
    m = len(constraints)

    for i, con in enumerate(constraints):
        artificial = [0.0] * m
        artificial[i] = 1.0
        con["coeffs"].extend(artificial)

    total_vars = nvars + m
    aux_objective = [0.0] * nvars + [1.0] * m

    tableau = [con["coeffs"] + [con["rhs"]] for con in constraints]
    w_row = [-sum(row[j] for row in tableau) for j in range(total_vars + 1)]
    tableau.append(w_row)

    basis = [f"r{i+1}" for i in range(m)]
    return {"tableau": tableau, "basis": basis, "objective": aux_objective, "nvars": total_vars, "m": m}

5. Реализация симплекс-метода

 - Вывод таблицы  
 - Поиск разрешающего элемента  
 - Один симплекс-шаг  
 - Основной цикл решения


In [ ]:
def print_table(tableau, basis):
    print("\nТекущая симплекс-таблица:")
    for i, row in enumerate(tableau):
        tag = basis[i] if i < len(basis) else " W "
        print(f"{tag:>3} |", "  ".join(f"{x:>8.3f}" for x in row))
    print()

def find_pivot(tableau):
    last_row = tableau[-1][:-1]
    pivot_col = np.argmin(last_row)
    if last_row[pivot_col] >= 0:
        return None, None

    ratios = []
    for i in range(len(tableau) - 1):
        aij = tableau[i][pivot_col]
        bi = tableau[i][-1]
        if aij > 0:
            ratios.append(bi / aij)
        else:
            ratios.append(float("inf"))

    if all(r == float("inf") for r in ratios):
        return -1, pivot_col

    pivot_row = int(np.argmin(ratios))
    return pivot_row, pivot_col

def pivot_step(tableau, basis, pivot_row, pivot_col):
    pivot_elem = tableau[pivot_row][pivot_col]
    tableau[pivot_row] = [x / pivot_elem for x in tableau[pivot_row]]

    for i in range(len(tableau)):
        if i != pivot_row:
            factor = tableau[i][pivot_col]
            tableau[i] = [tableau[i][j] - factor * tableau[pivot_row][j] for j in range(len(tableau[i]))]

    basis[pivot_row] = f"x{pivot_col + 1}"

def simplex_solve(aux_data):
    tableau = [row[:] for row in aux_data["tableau"]]
    basis = aux_data["basis"]

    print_table(tableau, basis)
    iteration = 1
    MAX_ITERS = 1000

    while iteration <= MAX_ITERS:
        pivot_row, pivot_col = find_pivot(tableau)
        if pivot_row is None:
            print("Вспомогательная задача решена.")
            return tableau, basis, True
        if pivot_row == -1:
            print("Вспомогательная задача не имеет решения (неограничена).")
            return tableau, basis, False

        print(f"Итерация {iteration}: разрешающий элемент ({pivot_row + 1}, {pivot_col + 1})")
        pivot_step(tableau, basis, pivot_row, pivot_col)
        print_table(tableau, basis)
        iteration += 1

    print("Превышено максимальное число итераций.")
    return tableau, basis, False

 6. Решение основной задачи


In [10]:
def transition_to_main_task(final_tableau, final_basis, original_obj, num_real_vars):
    tableau = [row[:num_real_vars] + [row[-1]] for row in final_tableau[:-1]]
    z_row = original_obj[:] + [0.0]

    for i, base_var in enumerate(final_basis):
        if base_var.startswith("x"):
            idx = int(base_var[1:]) - 1
            coef = original_obj[idx]
            for j in range(len(z_row)):
                z_row[j] -= coef * tableau[i][j]

    tableau.append(z_row)

    print("\nСимплекс-таблица основной задачи (начальная):")
    print_table(tableau, final_basis)
    return tableau, final_basis

def simplex_iterations(tableau, basis):
    iteration = 1
    MAX_ITERS = 1000

    while iteration <= MAX_ITERS:
        last_row = tableau[-1][:-1]
        pivot_col = np.argmin(last_row)
        if last_row[pivot_col] >= 0:
            print("\nОптимум найден!")
            return tableau, basis, True

        ratios = [tableau[i][-1] / tableau[i][pivot_col] if tableau[i][pivot_col] > 0 else float("inf")
                  for i in range(len(tableau) - 1)]
        if all(r == float("inf") for r in ratios):
            print("\nЗадача не имеет ограничений снизу — неограниченное решение.")
            return tableau, basis, False

        pivot_row = int(np.argmin(ratios))
        pivot_elem = tableau[pivot_row][pivot_col]

        print(f"\nИтерация {iteration}: разрешающий элемент ({pivot_row + 1}, {pivot_col + 1}) = {pivot_elem:.3f}")
        tableau[pivot_row] = [x / pivot_elem for x in tableau[pivot_row]]

        for i in range(len(tableau)):
            if i != pivot_row:
                factor = tableau[i][pivot_col]
                tableau[i] = [tableau[i][j] - factor * tableau[pivot_row][j] for j in range(len(tableau[i]))]

        basis[pivot_row] = f"x{pivot_col + 1}"
        print_table(tableau, basis)
        iteration += 1

    print("Превышено число итераций. Возможно, задача не имеет конечного оптимума.")
    return tableau, basis, False

def extract_solution(tableau, basis, num_vars, success, task_type="min"):
    x = [0.0] * num_vars
    for i, base_var in enumerate(basis):
        if base_var.startswith("x"):
            idx = int(base_var[1:]) - 1
            if idx < num_vars:
                x[idx] = tableau[i][-1]

    z = tableau[-1][-1]
    if task_type == "min" and z < 0:
        z = -z
    if not success:
        print("\nРешение не найдено — задача не имеет оптимального решения.")
    else:
        print("\n=== Итоговое решение ===")
        print("Оптимальные значения переменных:", x[:num_vars])
        print(f"Значение целевой функции: {z}")
    return x[:num_vars], z

 7. Основная программа



In [11]:
print("=== РЕШЕНИЕ ЗАДАЧИ ЛИНЕЙНОГО ПРОГРАММИРОВАНИЯ (ВАРИАНТ 18) ===\n")

# Создаём текстовый файл с задачей вариант 18
with open("task_variant18.txt", "w", encoding="utf-8") as f:
    f.write("""MIN
2 1 3 2
3
1 2 1 0 <= 11
1 0 1 1 = 8
0 1 0 1 >= 3
""")

# Создаём обновлённый Excel файл для варианта 18
excel_data = {
    'A': ['', 'целевая функция', '', '', '', 'ограничения', '', '', ''],
    'B': ['', '', 'x1', '1', '2', '', '1', '1', '0'],
    'C': ['', '', 'x2', '1', '1', '', '2', '0', '1'],
    'D': ['', '', 'x3', '1', '3', '', '1', '1', '0'],
    'E': ['', '', 'x4', '1', '2', '', '0', '1', '1'],
    'F': ['', '', 'Z', '=B5*B6+C5*C6+D5*D6+E5*E6', '', '', '=B8*B5+C8*C5+D8*D5+E8*E5', '=B9*B5+C9*C5+D9*D5+E9*E5', '=B10*B5+C10*C5+D10*D5+E10*E5'],
    'G': ['', '', '', '', '', '', '<=', '=', '>='],
    'H': ['', '', '', '', '', '', '11', '8', '3']
}

df = pd.DataFrame(excel_data)
df.to_excel('резултат_метопт.xlsx', index=False)
print("Файл Excel для варианта 18 создан: резултат_метопт.xlsx")

=== РЕШЕНИЕ ЗАДАЧИ ЛИНЕЙНОГО ПРОГРАММИРОВАНИЯ (ВАРИАНТ 18) ===

Файл Excel для варианта 18 создан: резултат_метопт.xlsx


Содержимое файла успешно прочитано:

{'constraints': [{'coeffs': [1.0, 2.0, 1.0, 0.0], 'rhs': 11.0, 'sense': '<='},
                 {'coeffs': [1.0, 0.0, 1.0, 1.0], 'rhs': 8.0, 'sense': '='},
                 {'coeffs': [0.0, 1.0, 0.0, 1.0], 'rhs': 3.0, 'sense': '>='}],
 'nvars': 4,
 'objective': [2.0, 1.0, 3.0, 2.0],
 'sense': 'min'}

После разбиения переменных (x = x+ - x-):

{'constraints': [{'coeffs': [1.0, -1.0, 2.0, -2.0, 1.0, -1.0, 0.0, -0.0],
                  'rhs': 11.0,
                  'sense': '<='},
                 {'coeffs': [1.0, -1.0, 0.0, -0.0, 1.0, -1.0, 1.0, -1.0],
                  'rhs': 8.0,
                  'sense': '='},
                 {'coeffs': [0.0, -0.0, 1.0, -1.0, 0.0, -0.0, 1.0, -1.0],
                  'rhs': 3.0,
                  'sense': '>='}],
 'nvars': 8,
 'objective': [2.0, -2.0, 1.0, -1.0, 3.0, -3.0, 2.0, -2.0],
 'sense': 'min',
 'var_mapping': ['x1+', 'x1-', 'x2+', 'x2-', 'x3+', 'x3-', 'x4+', 'x4-']}

Задача в каноническом виде:

{'constra